In [25]:
import pandas as pd
import numpy as np

from scipy.cluster.vq import kmeans, whiten

In [3]:
data_url = 'https://raw.githubusercontent.com/Thinkful-Ed/curric-data-001-data-sets/master/un/un.csv'

UN_data = pd.read_csv(data_url, header = 0)

In [11]:
UN_data.head()

,country,region,tfr,contraception,educationMale,educationFemale,lifeMale,lifeFemale,infantMortality,GDPperCapita,economicActivityMale,economicActivityFemale,illiteracyMale,illiteracyFemale
0,Afghanistan,Asia,6.90,NaN,NaN,NaN,45.0,46.0,154.0,2848.0,87.5,7.2,52.800,85.00
1,Albania,Europe,2.60,NaN,NaN,NaN,68.0,74.0,32.0,863.0,NaN,NaN,NaN,NaN
2,Algeria,Africa,3.81,52.0,11.1,9.9,67.5,70.3,44.0,1531.0,76.4,7.8,26.100,51.00
3,American.Samoa,Asia,NaN,NaN,NaN,NaN,68.0,73.0,11.0,NaN,58.8,42.4,0.264,0.36
4,Andorra,Europe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
print('The UN dataset contains %d rows.\n' % len(UN_data))

for col in UN_data.columns:
    print('Column %s has %d null values.' % (col, UN_data[col].isnull().sum()))
    
    print('Column %s contains a %s data type.\n' % (col, type(UN_data[col][0])))

The UN dataset contains 207 rows.

Column country has 0 null values.
Column country contains a <type 'str'> data type.

Column region has 0 null values.
Column region contains a <type 'str'> data type.

Column tfr has 10 null values.
Column tfr contains a <type 'numpy.float64'> data type.

Column contraception has 63 null values.
Column contraception contains a <type 'numpy.float64'> data type.

Column educationMale has 131 null values.
Column educationMale contains a <type 'numpy.float64'> data type.

Column educationFemale has 131 null values.
Column educationFemale contains a <type 'numpy.float64'> data type.

Column lifeMale has 11 null values.
Column lifeMale contains a <type 'numpy.float64'> data type.

Column lifeFemale has 11 null values.
Column lifeFemale contains a <type 'numpy.float64'> data type.

Column infantMortality has 6 null values.
Column infantMortality contains a <type 'numpy.float64'> data type.

Column GDPperCapita has 10 null values.
Column GDPperCapita contains

In [89]:
#Apply kMeans to subset of UN data
data_grp = ['lifeMale', 'lifeFemale', 'infantMortality', 'GDPperCapita']

UN_data = UN_data.dropna(subset = data_grp)

whitened_UN = whiten(np.array(UN_data[data_grp]))

UN_data['distance'] = -1
UN_data['centroid_code'] = ''

for code in range(1,2):
    centroids = kmeans(whitened_UN, code)
    
    for i, val in enumerate(centroids):
        for j in UN_data.index:
            print(j)
            UN_data['distance'][j] = pow(sum([pow(UN_data[grp][j] - val[0][k], 2) for k,grp in enumerate(data_grp)]), 0.5)
            UN_data['centroid_code'][j] = i    
                

0
1
2
5


C:\Users\danius.silkaitis\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\danius.silkaitis\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
50
52
53
54
55
56
57
58
59
60
61
63
64
65
67
68
69
70
72
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
108
109
111
112
113
114
115
116
117
118
119
120
121
122
123
124
126
127
128
129
130
131
132
133
134
135
136
137
138
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
162
163
164
165
166
167
168
169
170
171
172
173
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
191
192
193
194
195
196
197
198
199
200
203
204
205
206
0


IndexError: invalid index to scalar variable.

In [91]:
UN_data.index

Int64Index([  0,   1,   2,   5,   7,   8,   9,  10,  11,  12,
            ...
            195, 196, 197, 198, 199, 200, 203, 204, 205, 206],
           dtype='int64', length=188)